In [2]:
from datasets import load_dataset
dataset=load_dataset("yelp_review_full")


README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

C:\Users\samue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samue\.cache\huggingface\hub\datasets--yelp_review_full. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
print(dataset["train"][0])

{'label': 4, 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\samue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samue\.cache\huggingface\hub\models--google-bert--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets["train"][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 'input_ids': [101,
  173,
  1197,
  119,
  2284,
  2953,
  3272,
  1917,
  178,
  1440,
  1111,
  1107,
  170,
  1704,
  22351,
  119,
  1119,
  112,
  188,
  3505,
  1105,
  3123,
  1106,
  2037,
  1106,
  1443,
  1217,
  10063,
  4404,
  132,
  1119,
  112,
  188,
  1579,
  1113,
  1159,
  1107,
  3195,
  1117,
  4420,
  132,
  1119,
  112,
  188,
  6559,
  1114,
  170,
  1499,
  118,
  23555,
  2704,
  113,
  183,
  9379,
  114,
  1

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Full Model Fine-Tuning

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", logging_steps=5)

In [26]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.evaluate()

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.6290825605392456,
 'eval_model_preparation_time': 0.0015,
 'eval_accuracy': 0.196,
 'eval_runtime': 893.8143,
 'eval_samples_per_second': 1.119,
 'eval_steps_per_second': 0.14}

In [28]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 1.6909, 'grad_norm': 10.719744682312012, 'learning_rate': 4.933333333333334e-05, 'epoch': 0.04}
{'loss': 1.6032, 'grad_norm': 11.602723121643066, 'learning_rate': 4.866666666666667e-05, 'epoch': 0.08}
{'loss': 1.7458, 'grad_norm': 11.82161808013916, 'learning_rate': 4.8e-05, 'epoch': 0.12}
{'loss': 1.6001, 'grad_norm': 8.613147735595703, 'learning_rate': 4.7333333333333336e-05, 'epoch': 0.16}
{'loss': 1.671, 'grad_norm': 9.226383209228516, 'learning_rate': 4.666666666666667e-05, 'epoch': 0.2}
{'loss': 1.6995, 'grad_norm': 9.154727935791016, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.24}
{'loss': 1.6298, 'grad_norm': 8.690225601196289, 'learning_rate': 4.5333333333333335e-05, 'epoch': 0.28}
{'loss': 1.573, 'grad_norm': 9.406646728515625, 'learning_rate': 4.466666666666667e-05, 'epoch': 0.32}
{'loss': 1.5245, 'grad_norm': 6.542826175689697, 'learning_rate': 4.4000000000000006e-05, 'epoch': 0.36}
{'loss': 1.5254, 'grad_norm': 8.28404426574707, 'learning_rate': 4.333333333

TrainOutput(global_step=375, training_loss=1.0685885219573974, metrics={'train_runtime': 9375.005, 'train_samples_per_second': 0.32, 'train_steps_per_second': 0.04, 'total_flos': 789354427392000.0, 'train_loss': 1.0685885219573974, 'epoch': 3.0})

In [29]:
trainer.evaluate()

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.0107749700546265,
 'eval_accuracy': 0.598,
 'eval_runtime': 1397.7161,
 'eval_samples_per_second': 0.715,
 'eval_steps_per_second': 0.089,
 'epoch': 3.0}

In [63]:
trainer.evaluate(eval_dataset=small_train_dataset)

{'eval_loss': 0.5539393424987793,
 'eval_accuracy': 0.817,
 'eval_runtime': 899.1812,
 'eval_samples_per_second': 1.112,
 'eval_steps_per_second': 0.139,
 'epoch': 3.0}

In [30]:
# Save the fine-tuned model
model.save_pretrained('./fine-tuned-model')

In [31]:
from transformers import AutoModelForSequenceClassification

# Load the fine-tuned model
#model = AutoModelForSequenceClassification.from_pretrained('./fine-tuned-model')

In [ ]:
from huggingface_hub import login
login()
model.push_to_hub("HuggingfaceUsername/yourModelName")

In [64]:
import torch
import torch.nn.functional as F
import random

random_sample = dataset['train'][random.randint(0, len(dataset['train']) - 1)]
print(random_sample)
tt=tokenizer(random_sample["text"],return_tensors="pt", padding=True, truncation=True)
model.eval()

with torch.no_grad():
    outputs=model(**tt)

outputs

{'label': 4, 'text': 'Officially my favorite steakhouse hands down!  As if Vegas doesn\'t have enough to write about, this little gem is in one of the remaining standouts of what some call \\"Old Vegas\\".  The meal got off to a great start at the bar as we waited for our table.  The service was fast, the drinks were well made and naturally they were top shelf.  I was particularly pleased with the professional look and feel of the place.  Its very old school, mafioso, dark, white linen, with the sounds of clinking wine glasses and lively conversation in the background.  \\n\\nThe food:  Excellent!  Your steak is served as intended, well aged beef cooked to order on a hot plate...nothing else!  Fantastic.  The sides are of sharable portions so agree on one for every 2 or 3 people unless you plan on toting a box of leftovers around Vegas.  The mac and cheese is perfect as is the smooth and creamy lobster bisque.  The other members in my party had several cuts of meat and one had a chicke

SequenceClassifierOutput(loss=None, logits=tensor([[-1.7920, -1.4629, -1.2584,  0.7438,  3.3312]]), hidden_states=None, attentions=None)

In [65]:
import numpy as np

predictions = F.softmax(outputs.logits, dim=-1)
predictions

print("Prediction for random sample:" , (np.argmax(predictions)).item())

Prediction for random sample: 4


## PEFT (Parameter Efficient Fine-Tuning)

### LoRa

In [70]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, #RANK
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS 
)

In [73]:
#PRINT the number of trainable paraemters
def print_number_of_trainable_model_parameters(model):
    trainable_model_params=0
    all_model_params=0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        # If parameter isn't frozen
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model\n parameters:{trainable_model_params}\n all model parameters {all_model_params}\n percentrage of trainable model: {trainable_model_params/all_model_params*100}"

In [75]:
original_model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [76]:
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model
 parameters:1183493
 all model parameters 109497610
 percentrage of trainable model: 1.0808391160318476


In [82]:
peft_training_args = TrainingArguments(
    auto_find_batch_size=True,
    learning_rate= 1e-3,#higher learning rate than full fine-tuning
    num_train_epochs=3, #increase for more accuracy
    logging_steps=5,
    report_to = "none",
    output_dir="test_trainer",
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=small_train_dataset,
    compute_metrics=compute_metrics
)

In [83]:
peft_trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 1.7837, 'grad_norm': 8.398237228393555, 'learning_rate': 0.0009866666666666667, 'epoch': 0.04}
{'loss': 1.7265, 'grad_norm': 9.236401557922363, 'learning_rate': 0.0009733333333333334, 'epoch': 0.08}
{'loss': 1.6929, 'grad_norm': 8.746582984924316, 'learning_rate': 0.00096, 'epoch': 0.12}
{'loss': 1.6141, 'grad_norm': 4.345432758331299, 'learning_rate': 0.0009466666666666667, 'epoch': 0.16}
{'loss': 1.6893, 'grad_norm': 4.96002197265625, 'learning_rate': 0.0009333333333333333, 'epoch': 0.2}
{'loss': 1.6129, 'grad_norm': 5.919672966003418, 'learning_rate': 0.00092, 'epoch': 0.24}
{'loss': 1.5789, 'grad_norm': 8.735934257507324, 'learning_rate': 0.0009066666666666666, 'epoch': 0.28}
{'loss': 1.4992, 'grad_norm': 9.10274887084961, 'learning_rate': 0.0008933333333333333, 'epoch': 0.32}
{'loss': 1.3245, 'grad_norm': 3.858703136444092, 'learning_rate': 0.00088, 'epoch': 0.36}
{'loss': 1.2578, 'grad_norm': 4.778931617736816, 'learning_rate': 0.0008666666666666667, 'epoch': 0.4}
{'loss

TrainOutput(global_step=375, training_loss=1.0837730503082275, metrics={'train_runtime': 5904.4317, 'train_samples_per_second': 0.508, 'train_steps_per_second': 0.064, 'total_flos': 800261498880000.0, 'train_loss': 1.0837730503082275, 'epoch': 3.0})

In [85]:
# Save the LoRa fine-tuned model
peft_model.save_pretrained('./LoRa-fine-tuned-model')

In [84]:
peft_trainer.evaluate(eval_dataset=small_eval_dataset)

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.0099170207977295,
 'eval_accuracy': 0.565,
 'eval_runtime': 899.0478,
 'eval_samples_per_second': 1.112,
 'eval_steps_per_second': 0.139,
 'epoch': 3.0}

### QLoRa